In [1]:
a = 10

In [2]:
import pdfplumber
with pdfplumber.open("/home/dani/lstm_project/nextword/lstm-next-word-predictor/derma.pdf") as pdf:
    for i, page in enumerate(pdf.pages):
        text = page.extract_text()
        print(f"Page {i+1}")
        print(text)


Cannot set gray non-stroke color because /'P9' is an invalid float value
Cannot set gray non-stroke color because /'P15' is an invalid float value
Cannot set gray non-stroke color because /'P18' is an invalid float value


Page 1
Dermatology
British Association of Dermatologists
Page 2
Dermatology: Handbook for medical students & junior doctors
This publication is supported by the British Association of Dermatologists.
First edition 2009
Revised first edition 2009
Second edition 2014
Third edition 2020
For comments and feedback, please contact the author at drnicoledermatology@gmail.com
1 British Association of Dermatologists
Page 3
Dermatology: Handbook for medical students & junior doctors
Dermatology
Dr Nicole Yi Zhen Chiang MBChB (Hons), MRCP (UK), MRCP(UK)(Derm)
Consultant Dermatologist
Manchester University Hospitals NHS Trust
Withington Hospital M20 2LR
Manchester
Professor Julian Verbov JP MD FRCP FRCPCH CBiol FRSB FLS MCSFS
Professor of Dermatology
Consultant Paediatric Dermatologist
Alder Hey Children’s Hospital
East Prescot Road
Liverpool L14 5AB
2 British Association of Dermatologists
Page 4
Dermatology: Handbook for medical students & junior doctors
Contents
Preface 5
Foreword 5
What is derm

Cannot set gray non-stroke color because /'P341' is an invalid float value


Page 76
P
r
a
c
t
i c Dermatology: Handbook for medical students & junior doctors
a
l
S
k
i
l
l
s
– Prescribing emollients
C
lin
ic General tips
a
l
e ● Emollients come in 500 gram tubs
x
a
m
● In general, ointment-based emollients are useful for dry, scaling skin
in
a
t whereas creams and lotions are for red, inflamed and weeping lesions
io
n
a
n
d
in Prescribing topical corticosteroids
v
e
s General tips
t
ig
a
t ● Prescribe the weakest potency corticosteroid that is effective
io
n
s ● Use only for short term
● Need to specify the base i.e. cream, lotion or ointment
Clinical examination and investigations
Taking a skin swab
• Skin swabs can be taken from vesicles, pustules, erosions, ulcers and mucosal
surfaces for microbial culture.
• Surface swabs are generally not encouraged.
Taking a skin scrape
• Skin scrapes are taken from scaly lesions by gentle use of a scalpel in suspected
fungal infection (to show evidence of fungal hyphae and/or spores) and from
burrows in scabies (see pag

In [5]:
type(text)

str

In [15]:
import pdfplumber
text = ""
with pdfplumber.open("/home/dani/lstm_project/nextword/lstm-next-word-predictor/derma.pdf") as pdf:
    for i, page in enumerate(pdf.pages):
        text = text + page.extract_text()
        print(f"--- Page {i+1} ---")
        if text:
            print(text[:500])  # Print first 500 chars
        else:
            print("⚠️ No text found on this page.")


Cannot set gray non-stroke color because /'P9' is an invalid float value
Cannot set gray non-stroke color because /'P15' is an invalid float value
Cannot set gray non-stroke color because /'P18' is an invalid float value


--- Page 1 ---
Dermatology
British Association of Dermatologists
--- Page 2 ---
Dermatology
British Association of DermatologistsDermatology: Handbook for medical students & junior doctors
This publication is supported by the British Association of Dermatologists.
First edition 2009
Revised first edition 2009
Second edition 2014
Third edition 2020
For comments and feedback, please contact the author at drnicoledermatology@gmail.com
1 British Association of Dermatologists
--- Page 3 ---
Dermatology
British Association of DermatologistsDermatology: Handbook for medical students & junior doctors
This publication is supported by the British Association of Dermatologists.
First edition 2009
Revised first edition 2009
Second edition 2014
Third edition 2020
For comments and feedback, please contact the author at drnicoledermatology@gmail.com
1 British Association of DermatologistsDermatology: Handbook for medical students & junior doctors
Dermatology
Dr Nicole Yi Zhen Chiang MBChB (H
--- Page

Cannot set gray non-stroke color because /'P341' is an invalid float value


--- Page 78 ---
Dermatology
British Association of DermatologistsDermatology: Handbook for medical students & junior doctors
This publication is supported by the British Association of Dermatologists.
First edition 2009
Revised first edition 2009
Second edition 2014
Third edition 2020
For comments and feedback, please contact the author at drnicoledermatology@gmail.com
1 British Association of DermatologistsDermatology: Handbook for medical students & junior doctors
Dermatology
Dr Nicole Yi Zhen Chiang MBChB (H
--- Page 79 ---
Dermatology
British Association of DermatologistsDermatology: Handbook for medical students & junior doctors
This publication is supported by the British Association of Dermatologists.
First edition 2009
Revised first edition 2009
Second edition 2014
Third edition 2020
For comments and feedback, please contact the author at drnicoledermatology@gmail.com
1 British Association of DermatologistsDermatology: Handbook for medical students & junior doctors
Dermatology


In [16]:
len(text)

89322

In [19]:
text_list = text.split('\n')

In [23]:
import numpy as np 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout

In [24]:
print(text_list[0])  # Print first 10 lines to check the split

Dermatology


In [25]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_list)
total_words = len(tokenizer.word_index) + 1
input_sequences = []

for line in text_list:
    token_list_updated = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list_updated)):
        n_gram_sequence = token_list_updated[:i + 1]
        input_sequences.append(n_gram_sequence)

In [27]:
max_seq_len = max([len(x) for x in input_sequences])

In [28]:
input_sequences = pad_sequences(input_sequences, maxlen=max_seq_len, padding='pre')


In [29]:
x = input_sequences[:, :-1]
y = input_sequences[:, -1]


In [33]:
print(y[100])

24


In [34]:
len(y)

10734

In [35]:
y = to_categorical(y, num_classes=total_words)

In [36]:
model = Sequential()
model.add(Embedding(input_dim = total_words, output_dim = 10, input_length = max_seq_len - 1))
model.add(LSTM(150))
model.add(Dense(total_words, activation = 'softmax'))


/home/dani/lstm_project/nextword/venv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025-07-14 16:32:41.801223: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [37]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [38]:
model.fit(x, y, epochs = 10, verbose = 1)


Epoch 1/10


2025-07-14 16:33:15.763369: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 112105896 exceeds 10% of free system memory.


336/336 ━━━━━━━━━━━━━━━━━━━━ 23s 61ms/step - accuracy: 0.0321 - loss: 7.1315
Epoch 2/10
336/336 ━━━━━━━━━━━━━━━━━━━━ 22s 66ms/step - accuracy: 0.0390 - loss: 6.4891
Epoch 3/10
336/336 ━━━━━━━━━━━━━━━━━━━━ 22s 66ms/step - accuracy: 0.0532 - loss: 6.2647
Epoch 4/10
336/336 ━━━━━━━━━━━━━━━━━━━━ 40s 64ms/step - accuracy: 0.0986 - loss: 5.9913
Epoch 5/10
336/336 ━━━━━━━━━━━━━━━━━━━━ 23s 70ms/step - accuracy: 0.1129 - loss: 5.7406
Epoch 6/10
336/336 ━━━━━━━━━━━━━━━━━━━━ 23s 68ms/step - accuracy: 0.1320 - loss: 5.5482
Epoch 7/10
336/336 ━━━━━━━━━━━━━━━━━━━━ 22s 66ms/step - accuracy: 0.1430 - loss: 5.3246
Epoch 8/10
336/336 ━━━━━━━━━━━━━━━━━━━━ 23s 67ms/step - accuracy: 0.1497 - loss: 5.1331
Epoch 9/10
336/336 ━━━━━━━━━━━━━━━━━━━━ 24s 70ms/step - accuracy: 0.1622 - loss: 4.9559
Epoch 10/10
336/336 ━━━━━━━━━━━━━━━━━━━━ 22s 67ms/step - accuracy: 0.1706 - loss: 4.7649
